In [1]:
import ibis
import ibis.expr.datatypes as dt
import json
con = ibis.connect("duckdb://")
con.list_tables()

[]

In [2]:
con.attach_sqlite("Chinook_Sqlite.sqlite")
con.list_tables()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [49]:
tables = con.list_tables()
{table: con.table(table) for table in tables}

{'Album': DatabaseTable: Album
   AlbumId  int64
   Title    string
   ArtistId int64,
 'Artist': DatabaseTable: Artist
   ArtistId int64
   Name     string,
 'Customer': DatabaseTable: Customer
   CustomerId   int64
   FirstName    string
   LastName     string
   Company      string
   Address      string
   City         string
   State        string
   Country      string
   PostalCode   string
   Phone        string
   Fax          string
   Email        string
   SupportRepId int64,
 'Employee': DatabaseTable: Employee
   EmployeeId int64
   LastName   string
   FirstName  string
   Title      string
   ReportsTo  int64
   BirthDate  timestamp(6)
   HireDate   timestamp(6)
   Address    string
   City       string
   State      string
   Country    string
   PostalCode string
   Phone      string
   Fax        string
   Email      string,
 'Genre': DatabaseTable: Genre
   GenreId int64
   Name    string,
 'Invoice': DatabaseTable: Invoice
   InvoiceId         int64
   CustomerId  

In [1]:
import ibis
import json

def gvq_to_ibis(json_data):
    # Convert JSON to tables and joins
    tables = [node['label'] for node in json_data['nodes']]
    joins = [{edge['label'].split(' = ')[0]: edge['label'].split(' = ')[1]} for edge in json_data['edges']]
    
    # Connect to the DuckDB database
    con = ibis.connect("duckdb://")
    con.attach_sqlite("Chinook_Sqlite.sqlite")
    print('list_table:', con.list_tables())
    
    # Load tables into Ibis
    ibis_tables = {table: con.table(table) for table in tables}

    # Initialize a dictionary to hold the renamed columns for each table
    renamed_columns = {table: {} for table in tables}
    
    # Rename columns to unique names
    for table in tables:
        rename_dict = {f"{table}_{col}": col for col in ibis_tables[table].columns}
        renamed_columns[table] = {col: f"{table}_{col}" for col in ibis_tables[table].columns}
        ibis_tables[table] = ibis_tables[table].rename(rename_dict)
        # Debugging: Print the renamed columns to verify
        print(f"Renamed columns for table {table}: {ibis_tables[table].columns}")

    # Update join conditions to use renamed columns
    updated_joins = []
    for join in joins:
        updated_join = {}
        for left_table_col, right_table_col in join.items():
            left_table, left_col = left_table_col.split('.')
            right_table, right_col = right_table_col.split('.')
            try:
                updated_join[f"{renamed_columns[left_table][left_col]}"] = f"{renamed_columns[right_table][right_col]}"
            except KeyError as e:
                print(f"KeyError: {e} in join condition {join}")
                continue
        updated_joins.append(updated_join)
        # Debugging: Print the updated join conditions
        print(f"Updated join: {updated_join}")

    # Initialize a variable to hold the join expression
    join_expr = None

    for idx, join in enumerate(updated_joins):
        if not join:
            print(f"Skipping empty join condition at index {idx}")
            continue
        left_table_col, right_table_col = list(join.items())[0]
        left_table = left_table_col.split('_')[0]
        right_table = right_table_col.split('_')[0]

        if idx == 0:
            # First join
            join_expr = ibis_tables[left_table].join(
                ibis_tables[right_table],
                ibis_tables[left_table][left_table_col] == ibis_tables[right_table][right_table_col],
                how='left'
            )
        else:
            # Subsequent joins using the previous join result
            join_expr = join_expr.join(
                ibis_tables[right_table],
                join_expr[left_table_col] == ibis_tables[right_table][right_table_col],
                how='left'
            )

    return join_expr


In [2]:
# Test Case 1
json_data1 = {
  "nodes": [
    { "id": "1", "label": "Album", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "Artist", "position": { "x": 300, "y": 100 } }
  ],
  "edges": [
    { "source": "1", "target": "2", "label": "Album.ArtistId = Artist.ArtistId" }
  ]
}

json_data =json_data1

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result

list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table Album: ['Album_AlbumId', 'Album_Title', 'Album_ArtistId']
Renamed columns for table Artist: ['Artist_ArtistId', 'Artist_Name']
Updated join: {'Album_ArtistId': 'Artist_ArtistId'}
r0 := DatabaseTable: Album
  AlbumId  int64
  Title    string
  ArtistId int64

r1 := DatabaseTable: Artist
  ArtistId int64
  Name     string

r2 := Project[r0]
  Album_AlbumId:  r0.AlbumId
  Album_Title:    r0.Title
  Album_ArtistId: r0.ArtistId

r3 := Project[r1]
  Artist_ArtistId: r1.ArtistId
  Artist_Name:     r1.Name

JoinChain[r2]
  JoinLink[left, r3]
    r2.Album_ArtistId == r3.Artist_ArtistId
  values:
    Album_AlbumId:   r2.Album_AlbumId
    Album_Title:     r2.Album_Title
    Album_ArtistId:  r2.Album_ArtistId
    Artist_ArtistId: r3.Artist_ArtistId
    Artist_Name:     r3.Artist_Name


,Album_AlbumId,Album_Title,Album_ArtistId,Artist_ArtistId,Artist_Name
0,1,For Those About To Rock We Salute You,1,1,AC/DC
1,2,Balls to the Wall,2,2,Accept
2,3,Restless and Wild,2,2,Accept
3,4,Let There Be Rock,1,1,AC/DC
4,5,Big Ones,3,3,Aerosmith
...,...,...,...,...,...
342,325,Bartok: Violin & Viola Concertos,255,255,Yehudi Menuhin
343,343,Respighi:Pines of Rome,226,226,Eugene Ormandy
344,175,Walking Into Clarksdale,115,115,Page & Plant
345,253,"Battlestar Galactica (Classic), Season 1",158,158,Battlestar Galactica (Classic)


In [3]:
# Test Case 2
json_data2 = {
  "nodes": [
    { "id": "1", "label": "Invoice", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "Customer", "position": { "x": 300, "y": 100 } },
    { "id": "3", "label": "Employee", "position": { "x": 500, "y": 100 } }
  ],
  "edges": [
    { "source": "1", "target": "2", "label": "Invoice.CustomerId = Customer.CustomerId" },
    { "source": "2", "target": "3", "label": "Customer.SupportRepId = Employee.EmployeeId" }
  ]
}

json_data =json_data2

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result

list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table Invoice: ['Invoice_InvoiceId', 'Invoice_CustomerId', 'Invoice_InvoiceDate', 'Invoice_BillingAddress', 'Invoice_BillingCity', 'Invoice_BillingState', 'Invoice_BillingCountry', 'Invoice_BillingPostalCode', 'Invoice_Total']
Renamed columns for table Customer: ['Customer_CustomerId', 'Customer_FirstName', 'Customer_LastName', 'Customer_Company', 'Customer_Address', 'Customer_City', 'Customer_State', 'Customer_Country', 'Customer_PostalCode', 'Customer_Phone', 'Customer_Fax', 'Customer_Email', 'Customer_SupportRepId']
Renamed columns for table Employee: ['Employee_EmployeeId', 'Employee_LastName', 'Employee_FirstName', 'Employee_Title', 'Employee_ReportsTo', 'Employee_BirthDate', 'Employee_HireDate', 'Employee_Address', 'Employee_City', 'Employee_State', 'Employee_Country', 'Employee_PostalCode', 'Employee_Phone', 'Employee_F

,Invoice_InvoiceId,Invoice_CustomerId,Invoice_InvoiceDate,Invoice_BillingAddress,Invoice_BillingCity,Invoice_BillingState,Invoice_BillingCountry,Invoice_BillingPostalCode,Invoice_Total,Customer_CustomerId,...,Employee_BirthDate,Employee_HireDate,Employee_Address,Employee_City,Employee_State,Employee_Country,Employee_PostalCode,Employee_Phone,Employee_Fax,Employee_Email
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,...,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,...,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
2,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94,8,...,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,14,...,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86,23,...,1947-09-19,2003-05-03,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,357,50,2013-05-01,C/ San Bernardo 85,Madrid,None,Spain,28015,1.98,50,...,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
408,380,50,2013-08-03,C/ San Bernardo 85,Madrid,None,Spain,28015,3.96,50,...,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
409,400,44,2013-11-03,Porthaninkatu 9,Helsinki,None,Finland,00530,1.98,44,...,1973-08-29,2002-04-01,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
410,402,50,2013-11-05,C/ San Bernardo 85,Madrid,None,Spain,28015,5.94,50,...,1965-03-03,2003-10-17,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [4]:
# Test Case 3
json_data3 = {
  "nodes": [
    { "id": "1", "label": "InvoiceLine", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "Invoice", "position": { "x": 300, "y": 100 } },
    { "id": "3", "label": "Track", "position": { "x": 500, "y": 100 } },
    { "id": "4", "label": "Album", "position": { "x": 700, "y": 100 } }
  ],
  "edges": [
    { "source": "1", "target": "2", "label": "InvoiceLine.InvoiceId = Invoice.InvoiceId" },
    { "source": "1", "target": "3", "label": "InvoiceLine.TrackId = Track.TrackId" },
    { "source": "3", "target": "4", "label": "Track.AlbumId = Album.AlbumId" }
  ]
}

json_data =json_data3

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result

list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table InvoiceLine: ['InvoiceLine_InvoiceLineId', 'InvoiceLine_InvoiceId', 'InvoiceLine_TrackId', 'InvoiceLine_UnitPrice', 'InvoiceLine_Quantity']
Renamed columns for table Invoice: ['Invoice_InvoiceId', 'Invoice_CustomerId', 'Invoice_InvoiceDate', 'Invoice_BillingAddress', 'Invoice_BillingCity', 'Invoice_BillingState', 'Invoice_BillingCountry', 'Invoice_BillingPostalCode', 'Invoice_Total']
Renamed columns for table Track: ['Track_TrackId', 'Track_Name', 'Track_AlbumId', 'Track_MediaTypeId', 'Track_GenreId', 'Track_Composer', 'Track_Milliseconds', 'Track_Bytes', 'Track_UnitPrice']
Renamed columns for table Album: ['Album_AlbumId', 'Album_Title', 'Album_ArtistId']
Updated join: {'InvoiceLine_InvoiceId': 'Invoice_InvoiceId'}
Updated join: {'InvoiceLine_TrackId': 'Track_TrackId'}
Updated join: {'Track_AlbumId': 'Album_AlbumId'}
r0

,InvoiceLine_InvoiceLineId,InvoiceLine_InvoiceId,InvoiceLine_TrackId,InvoiceLine_UnitPrice,InvoiceLine_Quantity,Invoice_InvoiceId,Invoice_CustomerId,Invoice_InvoiceDate,Invoice_BillingAddress,Invoice_BillingCity,...,Track_AlbumId,Track_MediaTypeId,Track_GenreId,Track_Composer,Track_Milliseconds,Track_Bytes,Track_UnitPrice,Album_AlbumId,Album_Title,Album_ArtistId
0,1,1,2,0.99,1,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,...,2,2,1,None,342562,5510424,0.99,2,Balls to the Wall,2
1,2,1,4,0.99,1,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,...,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,3,Restless and Wild,2
2,3,2,6,0.99,1,2,4,2009-01-02,Ullevålsveien 14,Oslo,...,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,1,For Those About To Rock We Salute You,1
3,4,2,8,0.99,1,2,4,2009-01-02,Ullevålsveien 14,Oslo,...,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,1,For Those About To Rock We Salute You,1
4,6,2,12,0.99,1,2,4,2009-01-02,Ullevålsveien 14,Oslo,...,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,0.99,1,For Those About To Rock We Salute You,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,730,136,925,0.99,1,136,22,2010-08-15,120 S Orange Ave,Orlando,...,74,1,4,Bill Gould/Mike Patton,251663,8221247,0.99,74,Album Of The Year,82
2236,1125,208,3333,0.99,1,208,4,2011-06-29,Ullevålsveien 14,Oslo,...,259,1,15,None,221805,8874509,0.99,259,Radio Brasil (O Som da Jovem Vanguarda) - Sele...,36
2237,1697,313,3328,0.99,1,313,43,2012-10-06,"68, Rue Jouvence",Dijon,...,259,1,15,None,232881,9317533,0.99,259,Radio Brasil (O Som da Jovem Vanguarda) - Sele...,36
2238,1879,347,925,0.99,1,347,47,2013-03-05,"Via Degli Scipioni, 43",Rome,...,74,1,4,Bill Gould/Mike Patton,251663,8221247,0.99,74,Album Of The Year,82


In [5]:
# Example usage:
# Test Case 4
json_data4 = {
  "nodes": [
    { "id": "1", "label": "Track", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "Genre", "position": { "x": 300, "y": 100 } },
    { "id": "3", "label": "MediaType", "position": { "x": 500, "y": 100 } }
  ],
  "edges": [
    { "source": "1", "target": "2", "label": "Track.GenreId = Genre.GenreId" },
    { "source": "1", "target": "3", "label": "Track.MediaTypeId = MediaType.MediaTypeId" }
  ]
}

json_data =json_data4

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result

list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table Track: ['Track_TrackId', 'Track_Name', 'Track_AlbumId', 'Track_MediaTypeId', 'Track_GenreId', 'Track_Composer', 'Track_Milliseconds', 'Track_Bytes', 'Track_UnitPrice']
Renamed columns for table Genre: ['Genre_GenreId', 'Genre_Name']
Renamed columns for table MediaType: ['MediaType_MediaTypeId', 'MediaType_Name']
Updated join: {'Track_GenreId': 'Genre_GenreId'}
Updated join: {'Track_MediaTypeId': 'MediaType_MediaTypeId'}
r0 := DatabaseTable: Track
  TrackId      int64
  Name         string
  AlbumId      int64
  MediaTypeId  int64
  GenreId      int64
  Composer     string
  Milliseconds int64
  Bytes        int64
  UnitPrice    float64

r1 := DatabaseTable: Genre
  GenreId int64
  Name    string

r2 := DatabaseTable: MediaType
  MediaTypeId int64
  Name        string

r3 := Project[r0]
  Track_TrackId:      r0.TrackId
  

,Track_TrackId,Track_Name,Track_AlbumId,Track_MediaTypeId,Track_GenreId,Track_Composer,Track_Milliseconds,Track_Bytes,Track_UnitPrice,Genre_GenreId,Genre_Name,MediaType_MediaTypeId,MediaType_Name
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,1,Rock,1,MPEG audio file
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,1,Rock,2,Protected AAC audio file
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,1,Rock,2,Protected AAC audio file
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,1,Rock,2,Protected AAC audio file
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,1,Rock,2,Protected AAC audio file
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,24,Classical,2,Protected AAC audio file
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,24,Classical,2,Protected AAC audio file
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,24,Classical,2,Protected AAC audio file
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,24,Classical,2,Protected AAC audio file


In [6]:
# Test Case 5
json_data5 = {
  "nodes": [
    { "id": "1", "label": "Customer", "position": { "x": 100, "y": 100 } },
    { "id": "2", "label": "Invoice", "position": { "x": 300, "y": 100 } },
    { "id": "3", "label": "Employee", "position": { "x": 500, "y": 100 } },
    { "id": "4", "label": "Employee", "position": { "x": 700, "y": 100 } }  # Changed from 'Support' to 'Employee'
  ],
  "edges": [
    { "source": "1", "target": "2", "label": "Customer.CustomerId = Invoice.CustomerId" },
    { "source": "2", "target": "3", "label": "Invoice.SupportRepId = Employee.EmployeeId" },
    { "source": "1", "target": "4", "label": "Customer.SupportRepId = Employee.SupportRepId" }  # Adjusted the column names accordingly
  ]
}

json_data = json_data5

# Convert GVQ to Ibis expression
ibis_expr = gvq_to_ibis(json_data)
print(ibis_expr)

# Execute the Ibis expression and fetch results
result = ibis_expr.execute()
result

list_table: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Renamed columns for table Customer: ['Customer_CustomerId', 'Customer_FirstName', 'Customer_LastName', 'Customer_Company', 'Customer_Address', 'Customer_City', 'Customer_State', 'Customer_Country', 'Customer_PostalCode', 'Customer_Phone', 'Customer_Fax', 'Customer_Email', 'Customer_SupportRepId']
Renamed columns for table Invoice: ['Invoice_InvoiceId', 'Invoice_CustomerId', 'Invoice_InvoiceDate', 'Invoice_BillingAddress', 'Invoice_BillingCity', 'Invoice_BillingState', 'Invoice_BillingCountry', 'Invoice_BillingPostalCode', 'Invoice_Total']
Renamed columns for table Employee: ['Employee_EmployeeId', 'Employee_LastName', 'Employee_FirstName', 'Employee_Title', 'Employee_ReportsTo', 'Employee_BirthDate', 'Employee_HireDate', 'Employee_Address', 'Employee_City', 'Employee_State', 'Employee_Country', 'Employee_PostalCode', 'Employee_Phone', 'Employee_F

,Customer_CustomerId,Customer_FirstName,Customer_LastName,Customer_Company,Customer_Address,Customer_City,Customer_State,Customer_Country,Customer_PostalCode,Customer_Phone,...,Customer_SupportRepId,Invoice_InvoiceId,Invoice_CustomerId,Invoice_InvoiceDate,Invoice_BillingAddress,Invoice_BillingCity,Invoice_BillingState,Invoice_BillingCountry,Invoice_BillingPostalCode,Invoice_Total
0,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,...,5,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,...,4,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,...,4,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,...,4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,50,Enrique,Muñoz,None,C/ San Bernardo 85,Madrid,None,Spain,28015,+34 914 454 454,...,5,357,50,2013-05-01,C/ San Bernardo 85,Madrid,None,Spain,28015,1.98
408,50,Enrique,Muñoz,None,C/ San Bernardo 85,Madrid,None,Spain,28015,+34 914 454 454,...,5,380,50,2013-08-03,C/ San Bernardo 85,Madrid,None,Spain,28015,3.96
409,44,Terhi,Hämäläinen,None,Porthaninkatu 9,Helsinki,None,Finland,00530,+358 09 870 2000,...,3,400,44,2013-11-03,Porthaninkatu 9,Helsinki,None,Finland,00530,1.98
410,50,Enrique,Muñoz,None,C/ San Bernardo 85,Madrid,None,Spain,28015,+34 914 454 454,...,5,402,50,2013-11-05,C/ San Bernardo 85,Madrid,None,Spain,28015,5.94
